In [9]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import librosa
import glob 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


In [6]:
train = pd.read_csv('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/train.csv')

# Model 1

## Using multiple features

In [117]:
def feature_extractor(row):
    
    # defining path to train files
    file_name = os.path.join(os.path.abspath('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/'),
                            'Train', str(row.ID) + '.wav')
    
    if (row.ID%100 ==0):
        print ("Handling Row:", row.ID)
    
    #enclosing all this in try-catch to handle exceptions and missing files
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        
        #calculating MFCC feature
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=39).T,axis=0) 
        
        
        
        
        #calculating RMS feature for 400 frames
        from madmom.audio.signal import FramedSignal
        from madmom.audio.signal import root_mean_square

        fs = FramedSignal(file_name, num_channels=1,
                          num_frames=400)
        ##rounding rms values to the nearest integer 
        rms = np.around(root_mean_square(fs))
        
        
        
        
        #calculating ZCR feature
        from librosa.feature import zero_crossing_rate
        from librosa.core import load 

        ##padding with zeros to make a constant length of feature across all audios
        zcr_padded = np.zeros((210,))
        zcr = zero_crossing_rate(X, frame_length=2048, hop_length=441)
        zcr_padded[:zcr[0].shape[0]] = zcr[0]
        
        
        
        
        #calculating PSD feature
        from scipy import signal
        from madmom.audio.signal import Signal

        ##calculating PSD using Welch's method
        s = Signal(file_name, num_channels=1)
        f, P = signal.welch(s, 44100, nperseg=1024)
        
        
        

        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None, None

    features = np.concatenate ((mfccs, rms, zcr_padded, P))
    label = row.Class

    return [features, label]






In [118]:
#applying feature_estractor on each row in train 
new_train = train.apply(feature_extractor, axis=1)
new_train.columns = ['features', 'label']

Handling Row: 0
Handling Row: 0
Handling Row: 100
Handling Row: 200
Handling Row: 300
Handling Row: 400
Handling Row: 500
Handling Row: 600
Handling Row: 900
Handling Row: 1000
Handling Row: 1100
Handling Row: 1200
Handling Row: 1300
Handling Row: 1400
Handling Row: 1500
Handling Row: 1800
Handling Row: 2000
Handling Row: 2200
Handling Row: 2300
Handling Row: 2600
Handling Row: 2700
Handling Row: 2800
Handling Row: 2900
Handling Row: 3200
Handling Row: 3400
Handling Row: 3700
Handling Row: 3800
Handling Row: 3900
Handling Row: 4000
Handling Row: 4100
Handling Row: 4200
Handling Row: 4400
Handling Row: 4500
Handling Row: 4800
Handling Row: 4900
Handling Row: 5100
Handling Row: 5200
Handling Row: 5400
Handling Row: 5600
Handling Row: 5800
Handling Row: 5900
Handling Row: 6100
Handling Row: 6300
Handling Row: 6500
Handling Row: 6600
Handling Row: 6800
Handling Row: 6900
Handling Row: 7000
Handling Row: 7100
Handling Row: 7200
Handling Row: 7400
Handling Row: 7500
Handling Row: 7600
Handli

In [119]:
#dropping rows for missing files
new_train.dropna(inplace=True)
new_train.features[0].shape

(1162,)

In [120]:
#preparing data for feeding into the model
X = np.array(new_train.features.tolist())
y = np.array(new_train.label.tolist())

lb = LabelEncoder()

y = np_utils.to_categorical(lb.fit_transform(y))

num_labels = y.shape[1]


In [121]:
# building the model architecture

def create_model():

    model = Sequential()

    model.add(Dense(1024, input_shape=(1162,), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(512, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))

    
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    

    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    sgd= SGD(momentum=0.2, lr=0.03)


    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)

    return model
          
    


In [122]:
#building a pipeline and fitting it

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=200, batch_size=1000, verbose=1, )))
pipeline = Pipeline(estimators)

pipeline.fit(X,y)


Epoch 1/200
5435/5435 [==============================] - 4s 764us/step - loss: 3.2056 - acc: 0.1231
Epoch 2/200
5435/5435 [==============================] - 3s 613us/step - loss: 2.3533 - acc: 0.1603
Epoch 3/200
5435/5435 [==============================] - 3s 638us/step - loss: 2.2701 - acc: 0.1704
Epoch 4/200
5435/5435 [==============================] - 3s 546us/step - loss: 2.2123 - acc: 0.1950
Epoch 5/200
5435/5435 [==============================] - 3s 513us/step - loss: 2.1793 - acc: 0.2081
Epoch 6/200
5435/5435 [==============================] - 3s 602us/step - loss: 2.1613 - acc: 0.2075
Epoch 7/200
5435/5435 [==============================] - 3s 575us/step - loss: 2.1486 - acc: 0.2215
Epoch 8/200
5435/5435 [==============================] - 3s 542us/step - loss: 2.1145 - acc: 0.2280
Epoch 9/200
5435/5435 [==============================] - 3s 525us/step - loss: 2.0922 - acc: 0.2427
Epoch 10/200
5435/5435 [==============================] - 3s 582us/step - loss: 2.0952 - acc: 0.2355

5435/5435 [==============================] - 3s 464us/step - loss: 1.2086 - acc: 0.5649
Epoch 83/200
5435/5435 [==============================] - 3s 625us/step - loss: 1.2208 - acc: 0.5715
Epoch 84/200
5435/5435 [==============================] - 3s 530us/step - loss: 1.2076 - acc: 0.5772
Epoch 85/200
5435/5435 [==============================] - 3s 575us/step - loss: 1.2048 - acc: 0.5639
Epoch 86/200
5435/5435 [==============================] - 3s 554us/step - loss: 1.1927 - acc: 0.5748
Epoch 87/200
5435/5435 [==============================] - 3s 466us/step - loss: 1.1966 - acc: 0.5768
Epoch 88/200
5435/5435 [==============================] - 2s 451us/step - loss: 1.1881 - acc: 0.5810
Epoch 89/200
5435/5435 [==============================] - 2s 454us/step - loss: 1.1777 - acc: 0.5776
Epoch 90/200
5435/5435 [==============================] - 3s 462us/step - loss: 1.1737 - acc: 0.5842
Epoch 91/200
5435/5435 [==============================] - 2s 451us/step - loss: 1.1564 - acc: 0.5939
Epo

5435/5435 [==============================] - 3s 518us/step - loss: 0.8527 - acc: 0.6986
Epoch 163/200
5435/5435 [==============================] - 3s 483us/step - loss: 0.8514 - acc: 0.6984
Epoch 164/200
5435/5435 [==============================] - 2s 448us/step - loss: 0.8274 - acc: 0.7054
Epoch 165/200
5435/5435 [==============================] - 3s 471us/step - loss: 0.8318 - acc: 0.7128
Epoch 166/200
5435/5435 [==============================] - 3s 466us/step - loss: 0.8257 - acc: 0.7060
Epoch 167/200
5435/5435 [==============================] - 2s 457us/step - loss: 0.8291 - acc: 0.7141
Epoch 168/200
5435/5435 [==============================] - 2s 449us/step - loss: 0.7993 - acc: 0.7148
Epoch 169/200
5435/5435 [==============================] - 2s 457us/step - loss: 0.8433 - acc: 0.7078
Epoch 170/200
5435/5435 [==============================] - 2s 450us/step - loss: 0.8067 - acc: 0.7216
Epoch 171/200
5435/5435 [==============================] - 2s 456us/step - loss: 0.8108 - acc: 0

Pipeline(memory=None,
     steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasClassifier object at 0x1c4edda048>)])

In [45]:
#cross-validating pipeline
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(results.mean())

In [123]:
## creating a test dataframe

test = pd.read_csv('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/test.csv')

def feature_extractor_t(row):
    # function to load files and extract features
    file_name = os.path.join(os.path.abspath('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/'),
                            'Test', str(row.ID) + '.wav')
    if (row.ID%100 ==0):
        print ("Handling Row:", row.ID)

    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        
        #calculating MFCC feature
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=39).T,axis=0) 
        
        
        
        
        #calculating RMS feature for 400 frames
        from madmom.audio.signal import FramedSignal
        from madmom.audio.signal import root_mean_square

        fs = FramedSignal(file_name, num_channels=1,
                          num_frames=400)
        ##rounding rms values to the nearest integer 
        rms = np.around(root_mean_square(fs))
        
        
        
        
        #calculating ZCR feature
        from librosa.feature import zero_crossing_rate
        from librosa.core import load 

        ##padding with zeros to make a constant length of feature across all audios
        zcr_padded = np.zeros((210,))
        zcr = zero_crossing_rate(X, frame_length=2048, hop_length=441)
        zcr_padded[:zcr[0].shape[0]] = zcr[0]
        
        
        
        
        
        #calculating PSD feature
        from scipy import signal
        import matplotlib.pyplot as plt
        from madmom.audio.signal import Signal

        ##calculating PSD using Welch's method
        s = Signal(file_name, num_channels=1)
        f, P = signal.welch(s, 44100, nperseg=1024)
        
        
        



        
        


        
    except Exception as e:
        print("Error encountered while parsing file: ", e)
        return None

    features = np.concatenate ((mfccs, rms, zcr_padded, P))
    

    return features.tolist()

new_test = test.apply(feature_extractor_t, axis=1)
new_test.dropna(inplace=True)


Handling Row: 700
Handling Row: 800
Handling Row: 1600
Handling Row: 1700
Handling Row: 1900
Handling Row: 2100
Handling Row: 2400
Handling Row: 2500
Handling Row: 3000
Handling Row: 3100
Handling Row: 3300
Handling Row: 3500
Handling Row: 3600
Handling Row: 4300
Handling Row: 4600
Handling Row: 4700
Handling Row: 5000
Handling Row: 5300
Handling Row: 5500
Handling Row: 5700
Handling Row: 6000
Handling Row: 6200
Handling Row: 6400
Handling Row: 6700
Handling Row: 7300
Handling Row: 8400
Handling Row: 8600
Handling Row: 8700


In [125]:


X_t = np.array(new_test.tolist())

lb.inverse_transform

test['Class']=lb.inverse_transform(pipeline.predict(X_t))

test.to_csv('predictions_1_new.csv', index=False)




3297/3297 [==============================] - 1s 162us/step


# Model 2

## Using RMS as a feature

In [48]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import librosa
import glob 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


In [49]:
train = pd.read_csv('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/train.csv')

In [168]:
def feature_extractor(row):
    
    # defining path to train files
    file_name = os.path.join(os.path.abspath('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/'),
                            'Train', str(row.ID) + '.wav')
    if (row.ID%100 ==0):
        print ("Handling Row:", row.ID)

    
    #enclosing all this in try-catch to handle exceptions and missing files
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        

        
        
        
        #calculating RMS feature for 400 frames
        from madmom.audio.signal import FramedSignal
        from madmom.audio.signal import root_mean_square

        fs = FramedSignal(file_name, num_channels=1,
                          num_frames=400)
        ##rounding rms values to the nearest integer 
        rms = np.around(root_mean_square(fs))        
        

        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None, None

    features = rms
    label = row.Class

    return [features, label]






In [169]:
#applying feature_estractor on each row in train 
new_train = train.apply(feature_extractor, axis=1)
new_train.columns = ['features', 'label']

Handling Row: 0
Handling Row: 0
Handling Row: 100
Handling Row: 200
Handling Row: 300
Handling Row: 400
Handling Row: 500
Handling Row: 600
Handling Row: 900
Handling Row: 1000
Handling Row: 1100
Handling Row: 1200
Handling Row: 1300
Handling Row: 1400
Handling Row: 1500
Handling Row: 1800
Handling Row: 2000
Handling Row: 2200
Handling Row: 2300
Handling Row: 2600
Handling Row: 2700
Handling Row: 2800
Handling Row: 2900
Handling Row: 3200
Handling Row: 3400
Handling Row: 3700
Handling Row: 3800
Handling Row: 3900
Handling Row: 4000
Handling Row: 4100
Handling Row: 4200
Handling Row: 4400
Handling Row: 4500
Handling Row: 4800
Handling Row: 4900
Handling Row: 5100
Handling Row: 5200
Handling Row: 5400
Handling Row: 5600
Handling Row: 5800
Handling Row: 5900
Handling Row: 6100
Handling Row: 6300
Handling Row: 6500
Handling Row: 6600
Handling Row: 6800
Handling Row: 6900
Handling Row: 7000
Handling Row: 7100
Handling Row: 7200
Handling Row: 7400
Handling Row: 7500
Handling Row: 7600
Handli

In [170]:
#dropping rows for missing files
new_train.dropna(inplace=True)
new_train.features

0       [2230.0, 2711.0, 3081.0, 3126.0, 2970.0, 3176....
1       [5906.0, 6513.0, 7260.0, 7742.0, 7488.0, 7246....
2       [31.0, 32.0, 72.0, 72.0, 97.0, 97.0, 113.0, 10...
3       [4277.0, 4822.0, 5387.0, 5252.0, 4918.0, 5078....
4       [2028.0, 2490.0, 2755.0, 2442.0, 2822.0, 2554....
5       [266.0, 304.0, 350.0, 359.0, 367.0, 356.0, 363...
6       [462.0, 536.0, 604.0, 611.0, 601.0, 577.0, 557...
7       [2097.0, 2484.0, 2763.0, 2844.0, 2731.0, 2726....
8       [90.0, 107.0, 119.0, 7726.0, 9670.0, 10231.0, ...
9       [189.0, 223.0, 231.0, 233.0, 219.0, 214.0, 214...
10      [5457.0, 6106.0, 6391.0, 7235.0, 7726.0, 8298....
11      [1437.0, 1750.0, 2173.0, 2267.0, 2254.0, 2126....
12      [2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, ...
13      [4679.0, 4980.0, 6560.0, 6523.0, 7333.0, 7390....
14      [560.0, 668.0, 759.0, 779.0, 817.0, 782.0, 774...
15      [1684.0, 2108.0, 2532.0, 2879.0, 2832.0, 2880....
16      [2177.0, 2415.0, 2903.0, 2920.0, 3038.0, 3130....
17      [714.0

In [171]:
#preparing data for feeding into the model
X = np.array(new_train.features.tolist())
y = np.array(new_train.label.tolist())

lb = LabelEncoder()

y = np_utils.to_categorical(lb.fit_transform(y))

num_labels = y.shape[1]
X

array([[ 2230.,  2711.,  3081., ...,  2230.,  2102.,  1856.],
       [ 5906.,  6513.,  7260., ...,  8223.,  8203.,  8162.],
       [   31.,    32.,    72., ...,  3696.,  3508.,  3087.],
       ..., 
       [  143.,   212.,   239., ...,   229.,   231.,   227.],
       [  462.,   682.,   781., ...,  1253.,  1286.,  1238.],
       [ 4071.,  4891.,  5530., ...,  1723.,  1818.,  1681.]])

In [120]:
# building the model architecture

def create_model():

    model = Sequential()

    model.add(Dense(1024, input_shape=(400,), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(512, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))

    
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    

    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    sgd= SGD(momentum=0.2, lr=0.03)


    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)

    return model
          
    


In [121]:
#building a pipeline and fitting it

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=200, batch_size=1000, verbose=1, )))
pipeline = Pipeline(estimators)

pipeline.fit(X,y)


Epoch 1/200
5435/5435 [==============================] - 10s 2ms/step - loss: 2.8610 - acc: 0.1117
Epoch 2/200
5435/5435 [==============================] - 7s 1ms/step - loss: 2.4300 - acc: 0.1279
Epoch 3/200
5435/5435 [==============================] - 8s 1ms/step - loss: 2.3235 - acc: 0.1371
Epoch 4/200
5435/5435 [==============================] - 5s 971us/step - loss: 2.2930 - acc: 0.1419
Epoch 5/200
5435/5435 [==============================] - 4s 761us/step - loss: 2.2759 - acc: 0.1457
Epoch 6/200
5435/5435 [==============================] - 4s 736us/step - loss: 2.2456 - acc: 0.1557
Epoch 7/200
5435/5435 [==============================] - 4s 757us/step - loss: 2.2573 - acc: 0.1601
Epoch 8/200
5435/5435 [==============================] - 4s 782us/step - loss: 2.2264 - acc: 0.1647
Epoch 9/200
5435/5435 [==============================] - 5s 968us/step - loss: 2.2129 - acc: 0.1720
Epoch 10/200
5435/5435 [==============================] - 4s 754us/step - loss: 2.2082 - acc: 0.1661
Epoc

5435/5435 [==============================] - 2s 358us/step - loss: 1.8329 - acc: 0.3137
Epoch 163/200
5435/5435 [==============================] - 2s 357us/step - loss: 1.8372 - acc: 0.3102
Epoch 164/200
5435/5435 [==============================] - 2s 358us/step - loss: 1.8396 - acc: 0.3067
Epoch 165/200
5435/5435 [==============================] - 2s 357us/step - loss: 1.8310 - acc: 0.3091
Epoch 166/200
5435/5435 [==============================] - 2s 362us/step - loss: 1.8250 - acc: 0.3128
Epoch 167/200
5435/5435 [==============================] - 2s 361us/step - loss: 1.8414 - acc: 0.3045
Epoch 168/200
5435/5435 [==============================] - 2s 359us/step - loss: 1.8222 - acc: 0.3200
Epoch 169/200
5435/5435 [==============================] - 2s 363us/step - loss: 1.8363 - acc: 0.3176
Epoch 170/200
5435/5435 [==============================] - 2s 361us/step - loss: 1.8290 - acc: 0.3143
Epoch 171/200
5435/5435 [==============================] - 2s 362us/step - loss: 1.8200 - acc: 0

Pipeline(memory=None,
     steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasClassifier object at 0x1c4e6faf60>)])

In [58]:
#cross-validating pipeline
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(results.mean())

In [82]:
## creating a test dataframe

test = pd.read_csv('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/test.csv')

def feature_extractor_t(row):
    # function to load files and extract features
    file_name = os.path.join(os.path.abspath('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/'),
                            'Test', str(row.ID) + '.wav')
    if (row.ID%100 ==0):
        print ("Handling Row:", row.ID)

    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        

        
        #calculating RMS feature for 400 frames
        from madmom.audio.signal import FramedSignal
        from madmom.audio.signal import root_mean_square

        fs = FramedSignal(file_name, num_channels=1,
                          num_frames=400)
        ##rounding rms values to the nearest integer 
        rms = np.around(root_mean_square(fs))
        
        

        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None

    features = rms
    

    return features.tolist()

new_test = test.apply(feature_extractor_t, axis=1)
new_test.dropna(inplace=True)


Handling Row: 700
Handling Row: 800
Handling Row: 1600
Handling Row: 1700
Handling Row: 1900
Handling Row: 2100


KeyboardInterrupt: 

In [60]:


X_t = np.array(new_test.tolist())

lb.inverse_transform

test['Class']=lb.inverse_transform(pipeline.predict(X_t))

test.to_csv('predictions_2.csv', index=False)





3297/3297 [==============================] - 1s 217us/step


# Model 3

## Using ZCR feature


In [83]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import librosa
import glob 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


In [84]:
train = pd.read_csv('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/train.csv')

In [107]:
def feature_extractor(row):
    
    # defining path to train files
    file_name = os.path.join(os.path.abspath('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/'),
                            'Train', str(row.ID) + '.wav')
    if (row.ID%100 ==0):
        print ("Handling Row:", row.ID)
    
    #enclosing all this in try-catch to handle exceptions and missing files
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

        
        
        #calculating ZCR feature
        from librosa.feature import zero_crossing_rate
        from librosa.core import load 

        ##padding with zeros to make a constant length of feature across all audios
        zcr_padded = np.zeros((210,))
        zcr = zero_crossing_rate(X, frame_length=2048, hop_length=441)
        zcr_padded[:zcr[0].shape[0]] = zcr[0]
        zcr_padded

        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None, None

    features = zcr_padded
    label = row.Class

    return [features.tolist(), label]






In [108]:
#applying feature_estractor on each row in train 
new_train = train.apply(feature_extractor, axis=1)
new_train.columns = ['features', 'label']

Handling Row: 0
Handling Row: 0
Handling Row: 100
Handling Row: 200
Handling Row: 300
Handling Row: 400
Handling Row: 500
Handling Row: 600
Handling Row: 900
Handling Row: 1000
Handling Row: 1100
Handling Row: 1200
Handling Row: 1300
Handling Row: 1400
Handling Row: 1500
Handling Row: 1800
Handling Row: 2000
Handling Row: 2200
Handling Row: 2300
Handling Row: 2600
Handling Row: 2700
Handling Row: 2800
Handling Row: 2900
Handling Row: 3200
Handling Row: 3400
Handling Row: 3700
Handling Row: 3800
Handling Row: 3900
Handling Row: 4000
Handling Row: 4100
Handling Row: 4200
Handling Row: 4400
Handling Row: 4500
Handling Row: 4800
Handling Row: 4900
Handling Row: 5100
Handling Row: 5200
Handling Row: 5400
Handling Row: 5600
Handling Row: 5800
Handling Row: 5900
Handling Row: 6100
Handling Row: 6300
Handling Row: 6500
Handling Row: 6600
Handling Row: 6800
Handling Row: 6900
Handling Row: 7000
Handling Row: 7100
Handling Row: 7200
Handling Row: 7400
Handling Row: 7500
Handling Row: 7600
Handli

In [109]:
#dropping rows for missing files
new_train.dropna(inplace=True)
new_train.shape

(5435, 2)

In [111]:
#preparing data for feeding into the model
X = np.array(new_train.features.tolist())
y = np.array(new_train.label.tolist())

lb = LabelEncoder()

y = np_utils.to_categorical(lb.fit_transform(y))

num_labels = y.shape[1]
X[0]

array([ 0.04785156,  0.07226562,  0.09570312,  0.10742188,  0.10888672,
        0.11083984,  0.10888672,  0.10742188,  0.10498047,  0.10888672,
        0.10693359,  0.10058594,  0.09667969,  0.09570312,  0.09082031,
        0.09033203,  0.09033203,  0.09033203,  0.08984375,  0.09521484,
        0.09570312,  0.09960938,  0.09912109,  0.10058594,  0.10009766,
        0.10058594,  0.10302734,  0.09716797,  0.09912109,  0.10205078,
        0.09326172,  0.08837891,  0.08837891,  0.07666016,  0.07470703,
        0.07324219,  0.05615234,  0.06103516,  0.06640625,  0.06396484,
        0.07080078,  0.07861328,  0.07617188,  0.07568359,  0.08691406,
        0.08886719,  0.08935547,  0.09228516,  0.08984375,  0.08300781,
        0.08935547,  0.08789062,  0.08984375,  0.08691406,  0.09375   ,
        0.09716797,  0.09326172,  0.09423828,  0.09277344,  0.09179688,
        0.09130859,  0.09667969,  0.09423828,  0.09228516,  0.0859375 ,
        0.0859375 ,  0.07568359,  0.08056641,  0.08837891,  0.08

In [112]:
# building the model architecture

def create_model():

    model = Sequential()

    model.add(Dense(1024, input_shape=(210,), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(512, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))

    
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    

    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    sgd= SGD(momentum=0.2, lr=0.03)


    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)

    return model
          
    


In [113]:
#building a pipeline and fitting it

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=200, batch_size=1000, verbose=1, )))
pipeline = Pipeline(estimators)

pipeline.fit(X,y)


Epoch 1/200
5435/5435 [==============================] - 3s 494us/step - loss: 2.4551 - acc: 0.1338
Epoch 2/200
5435/5435 [==============================] - 2s 381us/step - loss: 2.2772 - acc: 0.1634
Epoch 3/200
5435/5435 [==============================] - 2s 340us/step - loss: 2.2137 - acc: 0.1820
Epoch 4/200
5435/5435 [==============================] - 2s 341us/step - loss: 2.2061 - acc: 0.1700
Epoch 5/200
5435/5435 [==============================] - 2s 339us/step - loss: 2.1785 - acc: 0.1856
Epoch 6/200
5435/5435 [==============================] - 2s 342us/step - loss: 2.1727 - acc: 0.1869
Epoch 7/200
5435/5435 [==============================] - 2s 339us/step - loss: 2.1435 - acc: 0.2057
Epoch 8/200
5435/5435 [==============================] - 2s 340us/step - loss: 2.1413 - acc: 0.2033
Epoch 9/200
5435/5435 [==============================] - 2s 339us/step - loss: 2.1367 - acc: 0.1928
Epoch 10/200
5435/5435 [==============================] - 2s 339us/step - loss: 2.1201 - acc: 0.1965

5435/5435 [==============================] - 2s 376us/step - loss: 1.8907 - acc: 0.2898
Epoch 83/200
5435/5435 [==============================] - 2s 452us/step - loss: 1.8858 - acc: 0.2997
Epoch 84/200
5435/5435 [==============================] - 2s 388us/step - loss: 1.8814 - acc: 0.2929
Epoch 85/200
5435/5435 [==============================] - 2s 387us/step - loss: 1.8798 - acc: 0.3016
Epoch 86/200
5435/5435 [==============================] - 2s 336us/step - loss: 1.8816 - acc: 0.2995
Epoch 87/200
5435/5435 [==============================] - 2s 336us/step - loss: 1.8784 - acc: 0.3005
Epoch 88/200
5435/5435 [==============================] - 2s 341us/step - loss: 1.8734 - acc: 0.2994
Epoch 89/200
5435/5435 [==============================] - 2s 334us/step - loss: 1.8754 - acc: 0.2983
Epoch 90/200
5435/5435 [==============================] - 2s 351us/step - loss: 1.8685 - acc: 0.3032
Epoch 91/200
5435/5435 [==============================] - 2s 343us/step - loss: 1.8669 - acc: 0.2949
Epo

5435/5435 [==============================] - 2s 361us/step - loss: 1.7904 - acc: 0.3255
Epoch 163/200
5435/5435 [==============================] - 2s 346us/step - loss: 1.7923 - acc: 0.3207
Epoch 164/200
5435/5435 [==============================] - 2s 339us/step - loss: 1.7868 - acc: 0.3255
Epoch 165/200
5435/5435 [==============================] - 2s 340us/step - loss: 1.7810 - acc: 0.3281
Epoch 166/200
5435/5435 [==============================] - 2s 334us/step - loss: 1.7816 - acc: 0.3255
Epoch 167/200
5435/5435 [==============================] - 2s 340us/step - loss: 1.7902 - acc: 0.3207
Epoch 168/200
5435/5435 [==============================] - 2s 339us/step - loss: 1.7799 - acc: 0.3257
Epoch 169/200
5435/5435 [==============================] - 2s 335us/step - loss: 1.7858 - acc: 0.3135
Epoch 170/200
5435/5435 [==============================] - 2s 335us/step - loss: 1.7860 - acc: 0.3273
Epoch 171/200
5435/5435 [==============================] - 2s 336us/step - loss: 1.7864 - acc: 0

Pipeline(memory=None,
     steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasClassifier object at 0x1c4890d160>)])

In [67]:
#cross-validating pipeline
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(results.mean())

In [116]:
## creating a test dataframe

test = pd.read_csv('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/test.csv')

def feature_extractor_t(row):
    # function to load files and extract features
    file_name = os.path.join(os.path.abspath('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/'),
                            'Test', str(row.ID) + '.wav')
    if (row.ID%100 ==0):
        print ("Handling Row:", row.ID)

    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        
        
        #calculating ZCR feature
        from librosa.feature import zero_crossing_rate
        from librosa.core import load 

        ##padding with zeros to make a constant length of feature across all audios
        zcr_padded = np.zeros((210,))
        zcr = zero_crossing_rate(X, frame_length=2048, hop_length=441)
        zcr_padded[:zcr[0].shape[0]] = zcr[0]
        zcr_padded


        
    except Exception as e:
        print("Error encountered while parsing file: ", e)
        return None

    features = zcr_padded
    

    return features.tolist()

new_test = test.apply(feature_extractor_t, axis=1)
new_test.dropna(inplace=True)


Handling Row: 700
Handling Row: 800
Handling Row: 1600
Handling Row: 1700
Handling Row: 1900
Handling Row: 2100
Handling Row: 2400
Handling Row: 2500
Handling Row: 3000
Handling Row: 3100
Handling Row: 3300
Handling Row: 3500
Handling Row: 3600
Handling Row: 4300
Handling Row: 4600
Handling Row: 4700
Handling Row: 5000
Handling Row: 5300
Handling Row: 5500
Handling Row: 5700
Handling Row: 6000
Handling Row: 6200
Handling Row: 6400
Handling Row: 6700
Handling Row: 7300
Handling Row: 8400
Handling Row: 8600
Handling Row: 8700


In [115]:


X_t = np.array(new_test.tolist())

lb.inverse_transform

test['Class']=lb.inverse_transform(pipeline.predict(X_t))

test.to_csv('predictions_3.csv', index=False)




3297/3297 [==============================] - 1s 158us/step


# Model 4

## Using PSD feature


In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import librosa
import glob 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


Using TensorFlow backend.


In [2]:
train = pd.read_csv('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/train.csv')

In [3]:
def feature_extractor(row):
    
    # defining path to train files
    file_name = os.path.join(os.path.abspath('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/'),
                            'Train', str(row.ID) + '.wav')
    if (row.ID%100 ==0):
        print ("Handling Row:", row.ID)
    
    #enclosing all this in try-catch to handle exceptions and missing files
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

        
        
        #calculating PSD feature
        from scipy import signal
        import matplotlib.pyplot as plt
        from madmom.audio.signal import Signal

        ##calculating PSD using Welch's method
        s = Signal(file_name, num_channels=1)
        f, P = signal.welch(s, 44100, nperseg=1024)
        

        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None, None

    features = P
    label = row.Class

    return [features, label]






In [4]:
#applying feature_estractor on each row in train 
new_train = train.apply(feature_extractor, axis=1)
new_train.columns = ['features', 'label']

Handling Row: 0
Handling Row: 0
Handling Row: 100
Handling Row: 200
Handling Row: 300
Handling Row: 400
Handling Row: 500
Handling Row: 600
Handling Row: 900
Handling Row: 1000
Handling Row: 1100
Handling Row: 1200
Handling Row: 1300
Handling Row: 1400
Handling Row: 1500
Handling Row: 1800
Handling Row: 2000
Handling Row: 2200
Handling Row: 2300
Handling Row: 2600
Handling Row: 2700
Handling Row: 2800
Handling Row: 2900
Handling Row: 3200
Handling Row: 3400
Handling Row: 3700
Handling Row: 3800
Handling Row: 3900
Handling Row: 4000
Handling Row: 4100
Handling Row: 4200
Handling Row: 4400
Handling Row: 4500
Handling Row: 4800
Handling Row: 4900
Handling Row: 5100
Handling Row: 5200
Handling Row: 5400
Handling Row: 5600
Handling Row: 5800
Handling Row: 5900
Handling Row: 6100
Handling Row: 6300
Handling Row: 6500
Handling Row: 6600
Handling Row: 6800
Handling Row: 6900
Handling Row: 7000
Handling Row: 7100
Handling Row: 7200
Handling Row: 7400
Handling Row: 7500
Handling Row: 7600
Handli

In [5]:
#dropping rows for missing files
new_train.dropna(inplace=True)
new_train.shape

(5435, 2)

In [6]:
#preparing data for feeding into the model
X = np.array(new_train.features.tolist())
y = np.array(new_train.label.tolist())

lb = LabelEncoder()

y = np_utils.to_categorical(lb.fit_transform(y))

num_labels = y.shape[1]
X

array([[  3.86222583e+03,   2.71326094e+04,   5.44416797e+04, ...,
          4.85184046e-06,   5.37353344e-06,   2.61063838e-06],
       [  2.51026025e+03,   8.72691602e+03,   2.61261660e+04, ...,
          4.33236528e-06,   4.08082587e-06,   1.83681414e-06],
       [  2.43888206e+01,   7.72827759e+01,   9.21221313e+01, ...,
          2.31134982e-05,   2.47511634e-05,   1.34520560e-05],
       ..., 
       [  4.73914642e+01,   2.42590179e+02,   1.93339615e+02, ...,
          1.02952309e-03,   9.46316228e-04,   4.66408586e-04],
       [  3.02349341e+03,   1.26310723e+04,   5.02322705e+03, ...,
          7.78507456e-06,   1.19406022e-05,   1.07108226e-05],
       [  7.73747754e+03,   5.44574219e+04,   5.64075703e+04, ...,
          7.99128175e-05,   8.25619281e-05,   3.87824657e-05]], dtype=float32)

In [7]:
# building the model architecture

def create_model():

    model = Sequential()

    model.add(Dense(1024, input_shape=(513,), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(512, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))

    
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    

    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    sgd= SGD(momentum=0.2, lr=0.03)


    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)

    return model
          
    


In [8]:
#building a pipeline and fitting it

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=200, batch_size=1000, verbose=1, )))
pipeline = Pipeline(estimators)

pipeline.fit(X,y)


Epoch 1/200
5435/5435 [==============================] - 3s 492us/step - loss: 2.4416 - acc: 0.1071
Epoch 2/200
5435/5435 [==============================] - 2s 408us/step - loss: 2.3382 - acc: 0.1178
Epoch 3/200
5435/5435 [==============================] - 2s 389us/step - loss: 2.2937 - acc: 0.1369
Epoch 4/200
5435/5435 [==============================] - 2s 366us/step - loss: 2.2733 - acc: 0.1468
Epoch 5/200
5435/5435 [==============================] - 2s 366us/step - loss: 2.2644 - acc: 0.1632
Epoch 6/200
5435/5435 [==============================] - 2s 365us/step - loss: 2.2675 - acc: 0.1588
Epoch 7/200
5435/5435 [==============================] - 2s 366us/step - loss: 2.2452 - acc: 0.1623
Epoch 8/200
5435/5435 [==============================] - 2s 366us/step - loss: 2.2168 - acc: 0.1777
Epoch 9/200
5435/5435 [==============================] - 2s 365us/step - loss: 2.2190 - acc: 0.1737
Epoch 10/200
5435/5435 [==============================] - 2s 365us/step - loss: 2.2043 - acc: 0.1805

5435/5435 [==============================] - 2s 366us/step - loss: 1.5335 - acc: 0.4686
Epoch 163/200
5435/5435 [==============================] - 2s 363us/step - loss: 1.5211 - acc: 0.4804
Epoch 164/200
5435/5435 [==============================] - 2s 368us/step - loss: 1.5163 - acc: 0.4771
Epoch 165/200
5435/5435 [==============================] - 2s 366us/step - loss: 1.4995 - acc: 0.4786
Epoch 166/200
5435/5435 [==============================] - 2s 364us/step - loss: 1.5123 - acc: 0.4775
Epoch 167/200
5435/5435 [==============================] - 2s 366us/step - loss: 1.5075 - acc: 0.4743
Epoch 168/200
5435/5435 [==============================] - 2s 365us/step - loss: 1.5100 - acc: 0.4732
Epoch 169/200
5435/5435 [==============================] - 2s 365us/step - loss: 1.4987 - acc: 0.4845
Epoch 170/200
5435/5435 [==============================] - 2s 364us/step - loss: 1.5063 - acc: 0.4791
Epoch 171/200
5435/5435 [==============================] - 2s 365us/step - loss: 1.5083 - acc: 0

Pipeline(memory=None,
     steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasClassifier object at 0x1c221e2d68>)])

In [9]:
#cross-validating pipeline
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(results.mean())

In [10]:
## creating a test dataframe

test = pd.read_csv('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/test.csv')

def feature_extractor_t(row):
    # function to load files and extract features
    file_name = os.path.join(os.path.abspath('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/'),
                            'Test', str(row.ID) + '.wav')
    if (row.ID%100 ==0):
        print ("Handling Row:", row.ID)

    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        
        
        #calculating PSD feature
        from scipy import signal
        import matplotlib.pyplot as plt
        from madmom.audio.signal import Signal

        ##calculating PSD using Welch's method
        s = Signal(file_name, num_channels=1)
        f, P = signal.welch(s, 44100, nperseg=1024)



        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None

    features = P
    

    return features.tolist()

new_test = test.apply(feature_extractor_t, axis=1)
new_test.dropna(inplace=True)


Handling Row: 700
Handling Row: 800
Handling Row: 1600
Handling Row: 1700
Handling Row: 1900
Handling Row: 2100
Handling Row: 2400
Handling Row: 2500
Handling Row: 3000
Handling Row: 3100
Handling Row: 3300
Handling Row: 3500
Handling Row: 3600
Handling Row: 4300
Handling Row: 4600
Handling Row: 4700
Handling Row: 5000
Handling Row: 5300
Handling Row: 5500
Handling Row: 5700
Handling Row: 6000
Handling Row: 6200
Handling Row: 6400
Handling Row: 6700
Handling Row: 7300
Handling Row: 8400
Handling Row: 8600
Handling Row: 8700


In [ ]:


X_t = np.array(new_test.tolist())

lb.inverse_transform

test['Class']=lb.inverse_transform(pipeline.predict(X_t))

test.to_csv('predictions_4.csv', index=False)




# Model 5

## Using MFCC feature


In [34]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import librosa
import glob 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


In [35]:
train = pd.read_csv('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/train.csv')

In [26]:
def feature_extractor(row):
    
    # defining path to train files
    file_name = os.path.join(os.path.abspath('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/'),
                            'Train', str(row.ID) + '.wav')
    
    #enclosing all this in try-catch to handle exceptions and missing files
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

        
        
        #calculating MFCC feature
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=39).T,axis=0) 
       

        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None, None

    features = mfccs
    label = row.Class

    return [features, label]






In [27]:
#applying feature_estractor on each row in train 
new_train = train.apply(feature_extractor, axis=1)
new_train.columns = ['features', 'label']

In [28]:
#dropping rows for missing files
new_train.dropna(inplace=True)
new_train.shape

(5435, 2)

In [38]:
#preparing data for feeding into the model
X = np.array(new_train.features.tolist())
y = np.array(new_train.label.tolist())

lb = LabelEncoder()

y = np_utils.to_categorical(lb.fit_transform(y))

num_labels = y.shape[1]


array([ -82.11494596,  139.47317581,  -42.41008515,   24.79492588,
        -11.59744647,   23.47668213,  -12.16276146,   25.87477093,
         -9.37786981,   21.19287823,   -7.34720007,   14.23773422,
         -8.65836673,    7.72666064,  -10.11617199,    3.2386625 ,
        -11.33468734,    2.78280142,   -7.03527193,    3.89992113,
         -2.32080626,    2.00126504,   -2.78118037,    4.11963711,
         -1.61454706,    4.31963577,   -1.02849855,   -1.24585996,
         -3.106575  ,    0.31221309,   -1.78428594,    0.43504187,
         -1.78772245,   -0.76478368,   -1.23643351,   -0.27920185,
          0.66139824,   -0.50357525,   -2.60634411])

In [32]:
# building the model architecture

def create_model():

    model = Sequential()

    model.add(Dense(1024, input_shape=(39,), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(512, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))

    
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    

    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    sgd= SGD(momentum=0.2, lr=0.03)


    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)

    return model
          
    


In [40]:
#building a pipeline and fitting it

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=200, batch_size=1000, verbose=1, )))
pipeline = Pipeline(estimators)

pipeline.fit(X,y)


Epoch 1/200
5435/5435 [==============================] - 3s 494us/step - loss: 2.3956 - acc: 0.0992
Epoch 2/200
5435/5435 [==============================] - 2s 452us/step - loss: 2.3010 - acc: 0.1321
Epoch 3/200
5435/5435 [==============================] - 3s 522us/step - loss: 2.2491 - acc: 0.1582
Epoch 4/200
5435/5435 [==============================] - 3s 485us/step - loss: 2.2069 - acc: 0.1862
Epoch 5/200
5435/5435 [==============================] - 2s 396us/step - loss: 2.1778 - acc: 0.2160
Epoch 6/200
5435/5435 [==============================] - 3s 468us/step - loss: 2.1368 - acc: 0.2405
Epoch 7/200
5435/5435 [==============================] - 3s 487us/step - loss: 2.0912 - acc: 0.2600
Epoch 8/200
5435/5435 [==============================] - 3s 466us/step - loss: 2.0616 - acc: 0.2826
Epoch 9/200
5435/5435 [==============================] - 2s 379us/step - loss: 2.0233 - acc: 0.2957
Epoch 10/200
5435/5435 [==============================] - 2s 345us/step - loss: 1.9861 - acc: 0.3174

5435/5435 [==============================] - 2s 430us/step - loss: 0.9241 - acc: 0.6857
Epoch 83/200
5435/5435 [==============================] - 2s 451us/step - loss: 0.9362 - acc: 0.6894
Epoch 84/200
5435/5435 [==============================] - 2s 427us/step - loss: 0.9091 - acc: 0.6955
Epoch 85/200
5435/5435 [==============================] - 2s 437us/step - loss: 0.9038 - acc: 0.6971
Epoch 86/200
5435/5435 [==============================] - 2s 439us/step - loss: 0.8987 - acc: 0.7019
Epoch 87/200
5435/5435 [==============================] - 2s 426us/step - loss: 0.9030 - acc: 0.6949
Epoch 88/200
5435/5435 [==============================] - 2s 418us/step - loss: 0.8901 - acc: 0.7056
Epoch 89/200
5435/5435 [==============================] - 2s 442us/step - loss: 0.8749 - acc: 0.7100
Epoch 90/200
5435/5435 [==============================] - 2s 450us/step - loss: 0.8809 - acc: 0.7017
Epoch 91/200
5435/5435 [==============================] - 3s 484us/step - loss: 0.8696 - acc: 0.7091
Epo

5435/5435 [==============================] - 2s 343us/step - loss: 0.6326 - acc: 0.7856
Epoch 163/200
5435/5435 [==============================] - 2s 325us/step - loss: 0.6172 - acc: 0.7899
Epoch 164/200
5435/5435 [==============================] - 2s 325us/step - loss: 0.6214 - acc: 0.7943
Epoch 165/200
5435/5435 [==============================] - 2s 325us/step - loss: 0.6207 - acc: 0.7932
Epoch 166/200
5435/5435 [==============================] - 2s 326us/step - loss: 0.6099 - acc: 0.7937
Epoch 167/200
5435/5435 [==============================] - 2s 325us/step - loss: 0.6118 - acc: 0.7996
Epoch 168/200
5435/5435 [==============================] - 2s 324us/step - loss: 0.6142 - acc: 0.7947
Epoch 169/200
5435/5435 [==============================] - 2s 321us/step - loss: 0.6073 - acc: 0.7961
Epoch 170/200
5435/5435 [==============================] - 2s 323us/step - loss: 0.6143 - acc: 0.7893
Epoch 171/200
5435/5435 [==============================] - 2s 325us/step - loss: 0.6053 - acc: 0

Pipeline(memory=None,
     steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasClassifier object at 0x1c3b52b630>)])

In [25]:
#cross-validating pipeline
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(results.mean())

Epoch 1/800
3409/3409 [==============================] - 2s 655us/step - loss: 3.2748 - acc: 0.1182
Epoch 2/800
3409/3409 [==============================] - 2s 455us/step - loss: 2.5730 - acc: 0.1496
Epoch 3/800
3409/3409 [==============================] - 2s 469us/step - loss: 2.3901 - acc: 0.1678
Epoch 4/800
3409/3409 [==============================] - 2s 452us/step - loss: 2.3325 - acc: 0.1725
Epoch 5/800
3409/3409 [==============================] - 2s 444us/step - loss: 2.2721 - acc: 0.1874
Epoch 6/800
3409/3409 [==============================] - 2s 450us/step - loss: 2.2343 - acc: 0.1874
Epoch 7/800
3409/3409 [==============================] - 2s 444us/step - loss: 2.1980 - acc: 0.2027
Epoch 8/800
3409/3409 [==============================] - 2s 451us/step - loss: 2.2054 - acc: 0.1968
Epoch 9/800
3409/3409 [==============================] - 2s 475us/step - loss: 2.1625 - acc: 0.2206
Epoch 10/800
3409/3409 [==============================] - 2s 488us/step - loss: 2.1344 - acc: 0.2309

KeyboardInterrupt: 

In [41]:
## creating a test dataframe

test = pd.read_csv('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/test.csv')

def feature_extractor_t(row):
    # function to load files and extract features
    file_name = os.path.join(os.path.abspath('/Users/pranjal/Desktop/audio_feature_extraction/case_study_data/'),
                            'Test', str(row.ID) + '.wav')

    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        
        
        #calculating MFCC feature
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=39).T,axis=0) 
       


        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None

    features = mfccs
    

    return features.tolist()

new_test = test.apply(feature_extractor_t, axis=1)
new_test.dropna(inplace=True)


In [42]:


X_t = np.array(new_test.tolist())

lb.inverse_transform

test['Class']=lb.inverse_transform(pipeline.predict(X_t))

test.to_csv('predictions_5.csv', index=False)




3297/3297 [==============================] - 1s 157us/step
